In [1]:
%pip install pydantic

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from pydantic import BaseModel, Field

class MyAssAddress(BaseModel):
    street: str = Field('default street', title="The street name", description="The street name", min_length=1, max_length=100, example="Main St.")
    city: str = Field('default city', title="The city name", description="The city name", min_length=1, max_length=100, example="Springfield")
    
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.street = ""
        self.city = ""
        
class MyAss(BaseModel):
    name: str = Field('default name', title="The name of the person", description="The name of the person", min_length=1, max_length=100, example="John Doe")
    address: MyAssAddress = Field(MyAssAddress(), title="The address of the person", description="The address of the person")
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.name = ""
        self.age = 0
        


In [20]:
from typing import Type, get_args, get_origin

def generate_json_schema(model_type: Type[BaseModel], indent: int = 4) -> str:
    lines = []
    indentation = " " * indent

    def process_model(model_type: Type[BaseModel], indent_level: int):
        current_indent = " " * indent_level
        lines.append(f"{current_indent}{{")
        for field_name, field in model_type.model_fields.items():
            field_type = field.annotation
            if get_origin(field_type) is list:  # Handling list/array of BaseModel
                item_type = get_args(field_type)[0]
                if issubclass(item_type, BaseModel):
                    lines.append(f'{current_indent}{indentation}"{field_name}": [')
                    process_model(item_type, indent_level + indent)
                    lines.append(f"{current_indent}{indentation}], # type: List[{item_type.__name__}], {field.description}")
                else:
                    lines.append(f'{current_indent}{indentation}"{field_name}": [...], # type: List[{item_type.__name__}], {field.description}')
            elif issubclass(field_type, BaseModel):  # Handling BaseModel field
                lines.append(f'{current_indent}{indentation}"{field_name}": ')
                process_model(field_type, indent_level + indent)
                lines.append(f"{current_indent}{indentation}, # type: {field_type.__name__}, {field.description}")
            else:
                lines.append(f'{current_indent}{indentation}"{field_name}": ..., # type: {field_type.__name__}, {field.description}')
        lines.append(f"{current_indent}}}")

    process_model(model_type, indent)
    return "\n".join(lines)

model_type: Type[BaseModel] = MyAss  # Replace with your actual model class
json_schema = generate_json_schema(model_type)
print(json_schema)

    {
        "name": ..., # type: str, The name of the person
    }


In [13]:
model_type.schema_json()

'{"properties": {"name": {"default": "default name", "description": "The name of the person", "example": "John Doe", "maxLength": 100, "minLength": 1, "title": "The name of the person", "type": "string"}}, "title": "MyAss", "type": "object"}'

In [56]:

from abc import ABC, abstractmethod

class Shape(ABC):
    x = 0
    y = 0
    color = "black"
    
    def __init__(self, x: int, y: int, color: str = "black"):
        self.x = x
        self.y = y
        self.color = color
    
    @abstractmethod
    def draw(self):
        pass
    
    @abstractmethod 
    def area(self):
        pass

class Rectangle(Shape):
    width = 0
    height = 0

    
    def __init__(self, x: int, y: int, width: int, height: int, color: str = "black"):
        super().__init__(x, y, color)
        self.width = width
        self.height = height

    def area(self):
        return self.calc_rectangle_area(self.width, self.height)

    def draw(self):
        print(f"Drawing with {self.color} at {self.x}, {self.y}")
        print(f"Rectangle Width: {self.width}, Height: {self.height}")
        

    @staticmethod
    def calc_rectangle_area(width: int, height: int):
        return width * height
        

class Square(Shape):
    def __init__(self, x: int, y: int, side: int, color: str = "black"):
        super().__init__(x, y, color)
        self.side = side
        
    def area(self):
        return self.side * self.side
    
    def draw(self):
        print(f"Drawing with {self.color} at {self.x}, {self.y}")
        print(f"Square Width: {self.side}")
 

In [62]:
myshape = Square(10, 20, 30)
myshape.draw()
myshape.side = 455
myshape.area()

Drawing with black at 10, 20
Square Side: 30


207025

In [65]:
   
class MyShape(Shape):
    def __init__(self, x: int, y: int, color: str = "black"):
        super().__init__(x, y, color)
    
    def area(self):
        return 0
    
    def draw(self):
        print(f"Drawing MyShape with {self.color} at {self.x}, {self.y}")
    
myshape = MyShape(10, 20, "green")
myshape.draw()

Drawing MyShape with green at 10, 20


In [68]:
shapes: list[Shape] = [Rectangle(1, 2, 3, 4, "red"), Square(1, 2, 3, "blue"), MyShape(1, 2, "green")]
for shape in shapes:    
    shape.draw()
   


Drawing with red at 1, 2
Rectangle Width: 3, Height: 4
Drawing with blue at 1, 2
Square Side: 3
Drawing MyShape with green at 1, 2
